In [ ]:
%cd /content/
%rm -rf LLaMA-Factory
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
%ls
!pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1
!pip uninstall -y jax
!pip install -e .[torch,bitsandbytes,liger-kernel]

In [ ]:
import torch
try:
  assert torch.cuda.is_available() is True
except AssertionError:
  print("GPU not found")

In [ ]:
import json

# using 404Dataset
%cd /content/LLaMA-Factory/

NAME = "Mus2VidLLaMA3"
AUTHOR = "Mus2Vid"

with open("404Dataset.json", "r", encoding="utf-8") as f:
  dataset = json.load(f)

with open("404Dataset.json", "w", encoding="utf-8") as f:
  json.dump(dataset, f, indent=2, ensure_ascii=False)


In [ ]:
%cd /content/LLaMA-Factory/
!GRADIO_SHARE=1 llamafactory-cli webui

In [ ]:
import json

args = dict(
  stage="sft",                        # Perform supervised fine-tuning
  do_train=True,
  model_name_or_path="unsloth/llama-3-8b-Instruct-bnb-4bit", # Use 4-bit quantized Llama-3-8b-Instruct model
  dataset="identity,alpaca_en_demo,alpaca_zh_demo",      # Use alpaca and self-awareness datasets
  template="llama3",                     # Use llama3 prompt template
  finetuning_type="lora",                   # Use LoRA adapter to save GPU memory
  lora_target="all",                     # Add LoRA adapter to all linear layers
  output_dir="llama3_lora",                  # Path to save LoRA adapter
  per_device_train_batch_size=2,               # Batch size
  gradient_accumulation_steps=4,               # Gradient accumulation steps
  lr_scheduler_type="cosine",                 # Use cosine learning rate decay
  logging_steps=10,                      # Output a log every 10 steps
  warmup_ratio=0.1,                      # Use learning rate warmup
  save_steps=1000,                      # Save a checkpoint every 1000 steps
  learning_rate=5e-5,                     # Learning rate
  num_train_epochs=3.0,                    # Number of training epochs
  max_samples=300,                      # Use 300 samples from each dataset
  max_grad_norm=1.0,                     # Clip gradient norm to 1.0
  loraplus_lr_ratio=16.0,                   # Use LoRA+ algorithm with lambda=16.0
  fp16=True,                         # Use float16 mixed precision training
  use_liger_kernel=True,                   # Use Liger Kernel to accelerate training
)


json.dump(args, open("train_llama3.json", "w", encoding="utf-8"), indent=2)

%cd /content/LLaMA-Factory/

!llamafactory-cli train train_llama3.json

In [ ]:
from llamafactory.chat import ChatModel
from llamafactory.extras.misc import torch_gc

%cd /content/LLaMA-Factory/

args = dict(
  model_name_or_path="unsloth/llama-3-8b-Instruct-bnb-4bit", # Use 4-bit quantized Llama-3-8b-Instruct model
  adapter_name_or_path="llama3_lora",            # Load previously saved LoRA adapter
  template="llama3",                     # Consistent with training
  finetuning_type="lora",                  # Consistent with training
  quantization_bit=4,                    # Load 4-bit quantized model
)
chat_model = ChatModel(args)

messages = []
print("Use `clear` to clear conversation history, use `exit` to exit the program.")
while True:
  query = input("\nUser: ")
  if query.strip() == "exit":
    break
  if query.strip() == "clear":
    messages = []
    torch_gc()
    print("Conversation history cleared")
    continue

  messages.append({"role": "user", "content": query})
  print("Assistant: ", end="", flush=True)

  response = ""
  for new_text in chat_model.stream_chat(messages):
    print(new_text, end="", flush=True)
    response += new_text
  print()
  messages.append({"role": "assistant", "content": response})

torch_gc()

In [ ]:
!huggingface-cli login

In [ ]:
import json

args = dict(
  model_name_or_path="meta-llama/Meta-Llama-3-8B-Instruct", # Use non-quantized official Llama-3-8B-Instruct model
  adapter_name_or_path="llama3_lora",            # Load previously saved LoRA adapter
  template="llama3",                     # Consistent with training
  finetuning_type="lora",                  # Consistent with training
  export_dir="llama3_lora_merged",              # Directory to save the merged model
  export_size=2,                       # Size of each weight file for the merged model (in GB)
  export_device="cpu",                    # Device used for merging model: `cpu` or `cuda`
  #export_hub_model_id="your_id/your_model",         # HuggingFace model ID for uploading the model
)

json.dump(args, open("merge_llama3.json", "w", encoding="utf-8"), indent=2)

%cd /content/LLaMA-Factory/

!llamafactory-cli export merge_llama3.json